# NLP basics

In [ ]:
import nltk

### Downloading the NLTK resources and models
The NLTK toolkit contains a lot of corpora (texts) which are useful for different tasks. It also contains all the pre-trained processing algorithms. If you are planning on working with it a lot and have enough hard drive space, then the best thing is simply to download everything from the downloader. The downloader is launched as follows:

In [ ]:
nltk.download()

Important note: you must close the downloader when finished otherwise it will block the notebook from executing further commands.

### Tokenizing

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize


In [ ]:
text = "This is a sentence. This is another sentence. Woo hoo!"

In [ ]:
stoks = sent_tokenize(text)

In [ ]:
stoks

### EXCERCISE 1: use the word_tokenizer to split the text into word tokens.

In [ ]:
wtoks = word_tokenize(text)

In [ ]:
wtoks

### Stripping

#### remove stopwords from a text

In [ ]:
text = "This is a very normal sentence but it has a lot of stopwords in it."

In [ ]:
nltk.download('stopwords')

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
toks = word_tokenize(text)

In [ ]:
[w for w in toks if w.lower() not in stopwords]

### Stemming

In [ ]:
stemmer = nltk.PorterStemmer()

In [ ]:
text = ["apples", "animals", "animation", "several"]

In [ ]:
[stemmer.stem(w) for w in text]

### EXCERCISE 2: There's another stemmer called the LancasterStemmer. Try it out and compare the results.

### Lemmatization

In [ ]:
nltk.download('wordnet')

In [ ]:
lem = nltk.WordNetLemmatizer()

In [ ]:
[lem.lemmatize(w) for w in text]

### POS Tagging

In [ ]:
from nltk.tag import pos_tag

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
nltk.pos_tag(wtoks)

### A reusable pre-processing function 

### Excercise 3: In the next examples, we're going to be doing a lot of initial preprocessing.
Write a reusable function called "preprocess" that takes a document and returns a list of tokenized sentences
Hint. First split it into sentences, then each sentence into words and then tag everything

In [ ]:
### Preprocess a document for NLP.
### Takes a plain text document as input
### Returns a list of a list of POS-tagged tuples

def preprocess(document):
    sentences = sent_tokenize(document)
    sentences = [word_tokenize(sent) for sent in sentences]
    sentences = [pos_tag(sent) for sent in sentences]
    return sentences

### Noun-phrase chunking

In [ ]:
text = """Software Development is such an important industry it is hard to imagine a world without it.
        Coal mining is also important, but it is usually a more dangerous job than software engineering."""

In [ ]:
### user-defined grammar using regular expression to chunk noun-phrases

grammar = ("NP: {"           ### label: every noun-phrase that we detect will get this label
           "<DT>?"           ### zero or one determiners, 
           "<JJ>*"           ### followed by an optional number of adjectives, 
           "<NN.*>+"         ### followed by any type or number of nouns
           "}")
                                        
                                        

#### (helpful app to develop regex grammars)

In [ ]:
nltk.app.chunkparser()

#### NP chunker

In [ ]:
### takes a pre-processed text, and a grammar (string) and return a result with the noun phrases "chunked"
def np_chunker(text, grammar):
    parser = nltk.RegexpParser(grammar)
    pp_text = preprocess(text)
    result = []
    for sent in pp_text:
        result.append(parser.parse(sent))
    return result

In [ ]:
chunked = np_chunker(text, grammar)

In [ ]:
chunked

### Excercise 4 write a function that takes in a chunked text and outputs only the noun phrases

Useful: http://stackoverflow.com/questions/14841997/how-to-navigate-a-nltk-tree-tree
   
Hint: for each sentence in a chunked text, iterate through the tags. If the label of the tag is NP then append it to the list (return a list of nodes).

In [ ]:
### solution:
def noun_phrase_extractor(chunked):
    noun_phrases = []
    for sentence in chunked:
        for n in sentence:
            if isinstance(n, nltk.tree.Tree):
                if n.label() == 'NP':
                    noun_phrases.append(n)
    return noun_phrases
        

In [ ]:
noun_phrase_extractor(chunked)

### Named entity recognition

In [ ]:
### in fact all the hard work has been done for us, with the standard ne_chunk function built into nltk

In [ ]:
text = """ERNI Consulting AG is a technology consulting company.
        The Chief Executive is Daniel Liechti and the Managing Director of Switzerland is Christoph Aeschlimann."""

In [ ]:
pp_text = preprocess(text)

In [ ]:
for sentence in pp_text:
        print(nltk.ne_chunk(sentence))

In [ ]:
for sentence in pp_text:
        print(nltk.ne_chunk(sentence).__str__)    ### workaround if you get an error above because ghostscript is not installed

# TextRazor demo

An API key for up to 500 free API calls per day is available: https://www.textrazor.com

## Text Classification

Preconditions:
1. Install beautifulsoup. e.g. pip install bs4
2. Install the textrazor python SDK: pip install textrazor (see https://github.com/TextRazor/textrazor-python)
3. Register for TextRazor and get an API key

In [ ]:
from textrazor import TextRazor
from bs4 import BeautifulSoup
import requests

In [ ]:
### get the wikipedia article on Donald Trump and use BeautifulSoup to extract only the text

url = 'https://en.wikipedia.org/wiki/Donald_Trump'
res = requests.get(url)
html = res.text
text = BeautifulSoup(html, 'lxml').get_text()

In [ ]:
### create and initialise the TextRazor client

In [ ]:
API_KEY = 'YOUR API KEY GOES HERE'

In [ ]:
client = TextRazor(API_KEY, extractors=["topics", "categories", "relations"])

In [ ]:
client.set_classifiers(['textrazor_newscodes'])

In [ ]:
response = client.analyze(text[8736:16000])      # the main article starts at 8736 chars into the text

In [ ]:
### assign the topics, categories and relations to variables for later use

In [ ]:
topics = list(response.topics())

In [ ]:
categories = list(response.categories())

In [ ]:
relations = list(response.relations())

In [ ]:
for topic in topics:
    if topic.score > 0.8:                     ### just the most relevant topics in the text
        print (topic.label)

In [ ]:
for category in categories:
    if category.score > 0.5:
        print (category.label, category.score)

## Relations detection

In [ ]:
client = TextRazor(API_KEY, extractors=["words", "entities", "entailments", "relations"])

In [ ]:
client.set_entity_freebase_type_filters(["/organization/organization"])
client.set_entity_dbpedia_type_filters(["Company"])

In [ ]:
text = "Accenture to acquire OCTO Technology within 6 months."

In [ ]:
response = client.analyze(text)

In [ ]:
relations = response.relations()

In [ ]:
### takes a list of words and returns the relations in the text
def get_relations(wordslist):
    relations_list = []
    for relation in relations:
        for word in relation.predicate_words:
            if word.lemma in wordslist:
                relations_list.append(relation)
                return relations_list

In [ ]:
buy_relations = get_relations(["sell", "buy", "acquire"])

In [ ]:
buy_relations

In [ ]:
for relation in buy_relations:
    entity_params = []
    for param in relation.params:
        all_entities = list(param.entities())

        if all_entities:
            entity_params.append(all_entities[0])

    if len(entity_params) > 1:
        print ("Found valid sell relationship between: ", entity_params)